# HBase / HappyBase Tutorial

In [41]:
import happybase

## Connecting via Thrift
The containerized HBase stack includes a trift server to make connecting to HBase simple.  Right now, the port is dynamic, so a `docker-compose ps` is needed to check the external thrift port.  Included in the listing of containers will be a line similar to: `thrift-1         entrypoint.sh thrift             Up      0.0.0.0:32776->9090/tcp`.  This indicats that port 9090 inside the container is mapped to port 32776 on localhost.  Below the connection is opened.

Note that the connection timeout is pretty low, so it might be necessary to reconnect.

In [114]:
connection = happybase.Connection('localhost', port=32776)

## Listing Tables
The tables that are in the database can be listed via the connection object.  Once listed, it is possible to connect to a table (if any exist).

In [44]:
connection.tables()

[b'test']

In [45]:
# Grab the table object
table = connection.table('test')

## Creating Tables
If a table does not already exist, it is possible to create a table.  Below, a table with 3 column families is created and the tables in the database are relisted.

In [52]:
try:
    connection.create_table(
    'mytable',
    {'cf1': dict(max_versions=10),
     'cf2': dict(max_versions=1, block_cache_enabled=False),
     'cf3': dict(),  # use defaults
    }
    )
except:
    print('Table already exists.')

Table already exists.


In [53]:
connection.tables()

[b'mytable', b'test']

## Inserting Data
HBase using binary storage for everything, so we have to do some `encode` / `decode` action on the Python side...

Here inserts happen row by row

In [115]:
table = connection.table('mytable')

In [104]:
import string
import random

table = connection.table('mytable')
rows = range(100)
for i in range(1000):
    rk = 'row{}'.format(random.choice(rows))
    cf1 = 'cf1:{}'.format(random.choice(['a', 'b', 'c']))
    cf2 = 'cf2:{}'.format(random.choice(['foo', 'bar', 'baz', 'zab', 'rab', 'oof']))
    cf3 = 'cf3:{}'.format(random.choice(['a', 'b', 'c', 'd', 'e', 'f', 'g']))
    v = '{}'.format(random.choice(string.ascii_letters)).encode()
    table.put(rk, {cf1:v, cf2:v, cf3:v})

In [110]:
table = connection.table('mytable')

for k, d in table.scan():
    print(k,d)
    #table.delete(k)

b'row0' {b'cf2:baz': b'u', b'cf3:d': b'p', b'cf2:bar': b'h', b'cf3:g': b'L', b'cf3:e': b'e', b'cf3:c': b'h', b'cf1:c': b'L', b'cf2:foo': b'i', b'cf1:a': b'e', b'cf3:b': b'u', b'cf2:rab': b'L', b'cf2:zab': b'e'}
b'row1' {b'cf1:b': b'c', b'cf1:c': b'q', b'cf3:d': b'q', b'cf2:zab': b'e', b'cf2:foo': b'V', b'cf1:a': b'V', b'cf3:b': b'V', b'cf2:bar': b'q', b'cf2:rab': b'c', b'cf3:e': b'c', b'cf3:g': b'e'}
b'row10' {b'cf2:baz': b'R', b'cf3:d': b'I', b'cf2:bar': b'I', b'cf3:g': b'R', b'cf1:b': b'R', b'cf3:c': b'U', b'cf2:oof': b'B', b'cf3:a': b'q', b'cf1:a': b'I', b'cf3:b': b'd', b'cf2:rab': b'd', b'cf1:c': b'U', b'cf3:f': b'a'}
b'row11' {b'cf3:d': b'M', b'cf2:baz': b'f', b'cf2:zab': b'S', b'cf3:g': b'S', b'cf3:c': b'f', b'cf1:b': b'x', b'cf3:e': b'V', b'cf2:oof': b'O', b'cf3:b': b'Y', b'cf1:a': b'M', b'cf2:foo': b'M', b'cf2:rab': b'D', b'cf1:c': b'f', b'cf3:f': b'D'}
b'row12' {b'cf3:c': b'M', b'cf1:b': b'v', b'cf1:c': b'M', b'cf2:baz': b'b', b'cf2:zab': b'h', b'cf1:a': b'h', b'cf3:d': b'j', 

In [111]:
# Cleaning up without deleting the table
for k, d in table.scan():
    table.delete(k)

## Batch Inserting Data
The above inserts are slow because everything is happening row by row.  In order to get the inserts to happen more quickly, it is possible to use batch operations.  This is possible using a context (`with table.batch() as b:`) or manually by calling a send.

In [112]:
table = connection.table('mytable')
rows = range(100)
batchsize = 1000
try:
    with table.batch(transaction=True) as b:
        for i in range(1000):
            rk = 'row{}'.format(random.choice(rows))
            cf1 = 'cf1:{}'.format(random.choice(['a', 'b', 'c']))
            cf2 = 'cf2:{}'.format(random.choice(['foo', 'bar', 'baz', 'zab', 'rab', 'oof']))
            cf3 = 'cf3:{}'.format(random.choice(['a', 'b', 'c', 'd', 'e', 'f', 'g']))
            v = '{}'.format(random.choice(string.ascii_letters)).encode()
            b.put(rk, {cf1:v, cf2:v, cf3:v})
except:
    print('An error occurred sending the batch')

In [113]:
for k, d in table.scan():
    print(k,d)

b'row0' {b'cf2:baz': b't', b'cf3:d': b'x', b'cf2:bar': b'q', b'cf3:g': b'q', b'cf1:b': b'x', b'cf3:e': b't', b'cf2:oof': b'x', b'cf3:a': b'u', b'cf2:foo': b't', b'cf1:a': b'T', b'cf3:b': b'T', b'cf2:rab': b'T', b'cf1:c': b't', b'cf3:f': b't'}
b'row1' {b'cf3:d': b'v', b'cf2:baz': b'v', b'cf2:zab': b'g', b'cf2:bar': b'V', b'cf3:g': b'g', b'cf3:c': b'K', b'cf1:b': b't', b'cf3:e': b'c', b'cf1:c': b'e', b'cf3:a': b'e', b'cf3:b': b'Y', b'cf1:a': b'v', b'cf2:foo': b'e', b'cf2:rab': b't'}
b'row10' {b'cf3:d': b'T', b'cf2:baz': b'v', b'cf2:zab': b'T', b'cf3:e': b'L', b'cf1:b': b'Z', b'cf3:c': b'Z', b'cf2:oof': b'X', b'cf3:a': b'P', b'cf2:foo': b'V', b'cf1:a': b'v', b'cf3:b': b'V', b'cf2:rab': b'P', b'cf1:c': b'T', b'cf3:f': b'v'}
b'row11' {b'cf3:d': b'd', b'cf2:baz': b'd', b'cf2:zab': b'n', b'cf2:bar': b'A', b'cf3:g': b'A', b'cf3:c': b'A', b'cf1:b': b'n', b'cf3:e': b'M', b'cf2:oof': b'A', b'cf3:a': b'Q', b'cf1:a': b'd', b'cf2:foo': b'p', b'cf2:rab': b'Q', b'cf1:c': b'A', b'cf3:f': b'd'}
b'row12'

In [116]:
# Cleaning up without deleting the table
for k, d in table.scan():
    table.delete(k)